# Boosting

---
## Experimental Setup

### Libraries & Settings

In [1]:
import os              # General OS commands
import numpy as np     # NumPy
import pandas as pd    # Python Data Analysis Library
import zipfile         # Compress/decompress ZIP files
import sqlite3         # SQLite3 Database Driver
import re              # Regular Expressions
import datetime        # Date/Time functions

In [2]:
from sklearn import ensemble, metrics, tree

In [3]:
# Never truncate columns, display all the data
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', -1)

# Display floating-point numbers with 4 decimals in `pandas.DataFrame`
pd.options.display.float_format = '{:,.4f}'.format

import matplotlib.pyplot as plt
# Display MatPlotLib stuff inline
%matplotlib inline

### Database

In [4]:
zip_filename = "../../data/ee-insee-2015_custom-sqlite.zip"
eedb = zip_filename.replace("-sqlite.zip", ".sqlite")

if not os.path.exists(eedb):
    with zipfile.ZipFile(zip_filename) as zip_file:
        zip_file.extractall("../../data/")

In [5]:
with sqlite3.connect(eedb) as con:
    query = "SELECT * FROM eec15_custom"
    eec15 = pd.read_sql_query(query, con)

In [6]:
# Create a ("female" x "enfant") interaction variable
bool_ = eec15[["enfants_", "female_"]].astype(bool)
eec15["female_enfants_"] = (bool_.enfants_ & bool_.female_).astype(int)

# Drop data we don't need
eec15 = eec15[eec15.age60_ == 0]
eec15 = eec15.drop("age60_", 1)

---
## Boosting


In [7]:
# Build a list of parameters to include in the model, using regex
# https://www.datarobot.com/blog/multiple-regression-using-statsmodels/
filters = {
    "age": "^age[0-9]{2}_$",
    "diploma": "^dip[0-9]{2}_$",
    "etranger": "^etranger_$",
    "domtom": "^domtom_$",
#     "trim": "^trim$",
    "female": "^female_$",
    "enfants": "^enfants_$",
#     "female_enfants": "^female_enfants_$",
    "region": "^region[1-2]_$"
}
params = {k: sorted([x for x in eec15.columns if re.match(r, x)]) for (k, r) in filters.items()}

# Avoid the dummy variable trap
params = {k: (v if len(v) == 1 else v[:-1]) for (k, v) in params.items()}
params["region"] += ["region2_"]

In [8]:
trims = ["t{}".format(x) for x in sorted(eec15["trim"].unique())]
eec15_ = {t: eec15[eec15.trim == int(t[1])] for t in trims}
X = {t: eec15_[t][sum(params.values(), [])] for t in eec15_}
y = {t: eec15_[t]["actop_"] for t in eec15_}

### Train the model using `trim=1` data

In [9]:
simple_tree_sk = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10,random_state=0).fit(X["t1"], y["t1"])
boost_sk = ensemble.AdaBoostClassifier(
    base_estimator=simple_tree_sk,
    n_estimators=100,
    learning_rate=0.1,
    random_state=0
).fit(X["t1"], y["t1"])

### Apply the model to test data (`trim=2/3/4`)

In [10]:
for t in trims:
#     predicted = random_forest_sk.predict_proba(X[t]).T[1]
#     predicted = np.where(predicted < 0.5, 0, 1)
    predicted = boost_sk.predict(X[t])
    print "Accuracy ({}): {}".format(t, metrics.accuracy_score(y[t], predicted))

# Note: the two methods for computing `predicted` are equivalent (THRESHOLD = 0.5)

Accuracy (t1): 0.777410143057
Accuracy (t2): 0.780638214565
Accuracy (t3): 0.772162940133
Accuracy (t4): 0.773814763805


In [11]:
for t in trims:
    predicted_proba = boost_sk.predict_proba(X[t]).T[1]
    print "ROC Area-Under-Curve ({}): {}".format(t, metrics.roc_auc_score(y[t], predicted_proba))

ROC Area-Under-Curve (t1): 0.820594780348
ROC Area-Under-Curve (t2): 0.818967354886
ROC Area-Under-Curve (t3): 0.809582476961
ROC Area-Under-Curve (t4): 0.812174478218


### Accuracy of prediction of `1` and `0`

In [12]:
for actop_ in [0, 1]:
    print "===== actop_ = {} =====".format(actop_)
    for t in trims:
        y_zero = y[t][y[t] == actop_]
        X_zero = X[t][X[t].index.map(lambda x: x in y_zero)]
    
        predicted = boost_sk.predict(X_zero)
        print "Accuracy ({}): {}".format(t, metrics.accuracy_score(y_zero, predicted))
    print

===== actop_ = 0 =====
Accuracy (t1): 0.916532497246
Accuracy (t2): 0.91491410111
Accuracy (t3): 0.912762233889
Accuracy (t4): 0.915493570636

===== actop_ = 1 =====
Accuracy (t1): 0.521221814777
Accuracy (t2): 0.527269791288
Accuracy (t3): 0.503742863185
Accuracy (t4): 0.507683504986



---
## Marginal Effects

###  By modifying the dataset ("passage de tout le monde en licence")

In [15]:
brute_force = pd.DataFrame()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1
        
        # Note mean() on the other dimension as compared to simple tree
        # This makes no difference on the mean(), but gives us the right std()
        proba_one = boost_sk.predict_proba(X_one).T[1]
        proba_t1 = boost_sk.predict_proba(X["t1"]).T[1]
        
        brute_force[pivot] = proba_one - proba_t1    # these are now vectors

In [16]:
marginal_effects = pd.DataFrame()
marginal_effects["brute_force_mean"] = brute_force.mean()
marginal_effects["brute_force_std"] = brute_force.std()
marginal_effects.transpose()

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_
brute_force_mean,0.0134,0.0116,-0.0133,-0.0223,-0.0369,-0.0996,-0.0611,-0.0279,-0.1678,-0.0023,-0.0107,0.0015,0.0251,0.0395,0.0096,-0.0196,0.0032,-0.0232,-0.0067
brute_force_std,0.1633,0.0950,0.0844,0.0823,0.1063,0.1817,0.2027,0.1060,0.2280,0.1114,0.0809,0.0753,0.1015,0.1839,0.0535,0.0843,0.0510,0.1434,0.1411


---
## Mean Marginal Effects (*bootstrap*)

We use the *bootstrap* method to calculate a confidence interval for our marginal effects. The *bootstrap* method consists in successive **random samplings with replacement** of the original database.

**Note: this is computationally very intensive – be careful or else**

### Computation

In [28]:
n = 5                             # number of iterations
k = eec15.index.size              # number of samples per iteration

In [29]:
index = sum(params.values(), [])
marginal_effects_iterations = pd.DataFrame(index=index)

start = datetime.datetime.now()
print "start: {}".format(start)

for i in range(n):  
    # take a sample of the DataFrame
    eec15_sample = eec15.sample(k, replace=True)

    # separate parameters and output
    X_sample = eec15_sample[index]
    y_sample = eec15_sample["actop_"]

    # build the boost for the sample
    sample_boost_sk = ensemble.AdaBoostClassifier(
        base_estimator=simple_tree_sk,
        n_estimators=100,
        learning_rate=0.1,
        random_state=0
    ).fit(X_sample, y_sample)

    # calculate marginal effects for that sample
    brute_force = pd.Series()
    for category in filters:
        for pivot in params[category]:
            non_pivots = [x for x in params[category] if x != pivot]
        
            X_one = X_sample.copy() 
            X_one[non_pivots] = 0
            X_one[pivot] = 1
        
            proba_one = sample_boost_sk.predict_proba(X_one).T[1].mean()
            proba_base = sample_boost_sk.predict_proba(X_sample).T[1].mean()
            brute_force[pivot] = proba_one - proba_base
    
    # store the results in the dataframe
    marginal_effects_iterations[i] = brute_force
    
    # print completed iteration
    print i+1,

print
end = datetime.datetime.now()
print "ran {} iterations in {}".format(n, end-start)

0 1 2 3 4


In [ ]:
marginal_effects_iterations.to_csv("csv/boosting_{}.csv".format(n))

### Mean/Standard Deviation (across iterations)

In [30]:
marginal_effects_summary = pd.DataFrame()
marginal_effects_summary["mean"] = marginal_effects_iterations.mean(axis=1)
marginal_effects_summary["std"] = marginal_effects_iterations.std(axis=1)

marginal_effects_summary.transpose()

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_
mean,0.0140,0.0147,-0.0143,-0.0204,-0.0348,-0.0775,-0.0446,-0.0224,-0.1092,-0.0007,-0.0108,0.0003,0.0216,0.0410,0.0086,-0.0190,0.0023,-0.0189,-0.0040
std,0.0033,0.0025,0.0016,0.0007,0.0012,0.0032,0.0078,0.0018,0.0053,0.0017,0.0010,0.0006,0.0011,0.0115,0.0001,0.0037,0.0002,0.0027,0.0013


### 95% Confidence Interval

In [17]:
interval_size = 0.99
alpha = (1 - interval_size)/2
ix = int(round(alpha*n))      # ix has to be an integer

interval = pd.DataFrame(index=["lower", "upper"])
for category in filters:
    for pivot in params[category]:
        me_pivot = marginal_effects_iterations.loc[pivot].sort_values()
        interval[pivot] = [me_pivot.iloc[ix], me_pivot.iloc[-ix]]

interval

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_
lower,0.0070,0.0104,-0.0128,-0.0202,-0.0350,-0.0827,-0.0610,-0.0259,-0.1073,0.0024,-0.0101,-0.0008,0.0211,0.0503,0.0085,-0.0206,0.0021,-0.0160,-0.0048
upper,0.0070,0.0104,-0.0128,-0.0202,-0.0350,-0.0827,-0.0610,-0.0259,-0.1073,0.0024,-0.0101,-0.0008,0.0211,0.0503,0.0085,-0.0206,0.0021,-0.0160,-0.0048


---
## Most important parameters
_ranked by **entropy importance**_ (**TODO**: where did Bastien get this?!)

In [17]:
entropy = pd.DataFrame(boost_sk.feature_importances_, index=X["t1"].columns, columns=["entropy"])
#entropy_sorted = entropy.sort_values(by="entropy", ascending=False).transpose()
entropy.transpose()

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_
entropy,0.0647,0.1263,0.0583,0.0404,0.0425,0.0195,0.0128,0.0286,0.0193,0.0388,0.0698,0.0693,0.0602,0.0104,0.0616,nan,0.0532,0.0498,0.0800


---
# Odds Ratios

The odds ratios formula is:
$$\frac{p/(1-p)}{q/(1-q)} = \frac{p(1-q)}{q(1-p)}$$

where $p$ and $q$ are the probability of being **unemployed**.

In [16]:
odds = pd.Series()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1

        proba_one = boost_sk.predict_proba(X_one).T[1].mean()
        odds[pivot] = proba_one/(1-proba_one)  

In [17]:
odds_ratios = pd.DataFrame(index=odds.index, columns=odds.index)
for col in odds.index:
    for row in odds.index:
        odds_ratios[col][row] = odds[col]/odds[row]

In [18]:
# odds_ratios     # col, row = odds[col]/odds[row]
odds_ratios.T   # row, col = odds[row]/odds[col]  

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_
etranger_,1.0000,1.0070,1.1130,1.1539,1.2241,1.5873,1.3514,1.1803,2.1494,1.0647,1.1014,1.0489,0.9542,0.9009,1.0152,1.1415,1.0415,1.1580,1.0838
age15_,0.9931,1.0000,1.1052,1.1459,1.2156,1.5763,1.3420,1.1721,2.1345,1.0573,1.0937,1.0416,0.9476,0.8946,1.0082,1.1336,1.0343,1.1500,1.0763
age30_,0.8985,0.9048,1.0000,1.0368,1.0999,1.4262,1.2142,1.0605,1.9313,0.9567,0.9896,0.9424,0.8574,0.8094,0.9122,1.0256,0.9358,1.0405,0.9738
age40_,0.8666,0.8727,0.9645,1.0000,1.0608,1.3756,1.1711,1.0228,1.8627,0.9227,0.9545,0.9090,0.8269,0.7807,0.8798,0.9892,0.9026,1.0036,0.9392
dip10_,0.8169,0.8226,0.9092,0.9427,1.0000,1.2967,1.1040,0.9642,1.7559,0.8698,0.8997,0.8569,0.7795,0.7359,0.8294,0.9325,0.8508,0.9460,0.8854
dip11_,0.6300,0.6344,0.7012,0.7270,0.7712,1.0000,0.8514,0.7436,1.3542,0.6708,0.6939,0.6608,0.6012,0.5675,0.6396,0.7191,0.6561,0.7296,0.6828
dip30_,0.7400,0.7452,0.8236,0.8539,0.9058,1.1746,1.0000,0.8734,1.5905,0.7879,0.8150,0.7761,0.7061,0.6666,0.7512,0.8447,0.7707,0.8569,0.8020
dip31_,0.8473,0.8532,0.9430,0.9777,1.0371,1.3448,1.1450,1.0000,1.8211,0.9021,0.9331,0.8887,0.8085,0.7633,0.8602,0.9671,0.8824,0.9811,0.9182
dip33_,0.4652,0.4685,0.5178,0.5369,0.5695,0.7385,0.6287,0.5491,1.0000,0.4953,0.5124,0.4880,0.4439,0.4191,0.4723,0.5311,0.4845,0.5388,0.5042
dip41_,0.9392,0.9458,1.0453,1.0838,1.1497,1.4908,1.2693,1.1086,2.0188,1.0000,1.0344,0.9851,0.8962,0.8461,0.9535,1.0721,0.9782,1.0877,1.0179


---
## Odds Ratios (2)
**TODO**: add the other Odds Ratios formula

In [21]:
odds_ratios_2 = pd.DataFrame()
for category in filters:
    for pivot in params[category]:
        non_pivots = [x for x in params[category] if x != pivot]
        
        X_one = X["t1"].copy() 
        X_one[non_pivots] = 0
        X_one[pivot] = 1
        proba_one = boost_sk.predict_proba(X_one).T[1].mean()
        odds_one = proba_one/(1-proba_one)
        
        X_zero = X["t1"].copy()
        X_zero[non_pivots] = 0
        X_zero[pivot] = 0
        proba_zero = boost_sk.predict_proba(X_zero).T[1].mean()
        odds_zero = proba_zero/(1-proba_zero)
        
        odds_ratios_2.loc[pivot, "ratio"] = odds_one/odds_zero

odds_ratios_2.transpose()

,etranger_,age15_,age30_,age40_,dip10_,dip11_,dip30_,dip31_,dip33_,dip41_,dip42_,dip50_,dip60_,dip70_,female_,enfants_,region1_,region2_,domtom_
ratio,1.0599,1.1123,1.0064,0.9707,0.7706,0.5942,0.6980,0.7992,0.4388,0.8859,0.8564,0.8993,0.9885,1.0470,1.1075,0.9081,1.1598,1.0431,0.9773
